In [ ]:
import telebot
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
bot = telebot.TeleBot("7687972629:AAFR7GbE044QQls156eZv6L5CUaibXeR1Cw", parse_mode="HTML")
model = load_model('/content/drive/MyDrive/model.h5')

In [10]:
flower_dict = {
    0: "Bluebell",
    1: "Buttercup",
    2: "Coltsfoot",
    3: "Cowslip",
    4: "Crocus",
    5: "Daffodil",
    6: "Daisy",
    7: "Dandelion",
    8: "Fritillary",
    9: "Iris",
    10: "Lily Valley",
    11: "Pansy",
    12: "Snowdrop",
    13: "Sunflower",
    14: "Tigerlily",
    15: "Tulip",
    16: "Windflower"
}

datagen = ImageDataGenerator(rescale=1./255)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, f"Hi {message.from_user.first_name}! 🌸 Send me a flower picture to identify it.")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    with open("image.jpg", "wb") as file:
        file.write(downloaded_file)

    bot.reply_to(message, "🔄 Processing the image...")

    image = load_img("image.jpg", target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    image_gen = datagen.flow(image, batch_size=1)
    image_processed = next(image_gen)

    output = model.predict(image_processed)
    predict = np.argmax(output)
    flower_name = flower_dict.get(predict, "Unknown Flower")

    bot.reply_to(message,f"🌺 This looks like a <b>{flower_name}</b>!")

bot.polling()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
